In [11]:
# Libraries, Variables and Constants

import autogen, os
import json
from dotenv import load_dotenv

load_dotenv("./../credentials_my.env")
cache_seed = 41 # default seed is 41

In [12]:
my_models = [
    {
        'model': os.environ['GPT4-0125PREVIEW-128k'],
        'api_key': os.environ['AZURE_OPENAI_API_KEY_SCUS'],
        "base_url": os.environ['AZURE_OPENAI_ENDPOINT_SCUS'],
        "api_type": os.environ['OPENAI_API_TYPE'],
        "api_version": os.environ['AZURE_OPENAI_API_VERSION']
    },
    {
        'model': os.environ['GPT4-1106-128k'],
        'api_key': os.environ['AZURE_OPENAI_API_KEY'],
        "base_url": os.environ['AZURE_OPENAI_ENDPOINT'],
        "api_type": os.environ['OPENAI_API_TYPE'],
        "api_version": os.environ['AZURE_OPENAI_API_VERSION']
    },
    {
        'model': os.environ['GPT432-0613-32k'],
        'api_key': os.environ['AZURE_OPENAI_API_KEY'],
        "base_url": os.environ['AZURE_OPENAI_ENDPOINT'],
        "api_type": os.environ['OPENAI_API_TYPE'],
        "api_version": os.environ['AZURE_OPENAI_API_VERSION']
    }
]

# need for converting to string
os.environ['models_var'] = json.dumps(my_models)

# Setting configurations for autogen
config_list = autogen.config_list_from_json(
    env_or_file='models_var',
    filter_dict={
        "model": {"gpt4-0125preview-128k", "gpt-4"}
    }
)

# config_list

In [13]:
# Setting configurations for autogen
config_list = autogen.config_list_from_json(
    env_or_file='models_list.json',
    filter_dict={ "model": {"gpt4-0125preview-128k", "gpt-4"} }
)

# config_list

In [14]:
llm_config={
        "cache_seed": cache_seed,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API configurations
        "temperature": 0,  # temperature for sampling
    }

# llm_config

In [15]:
# create a UserProxyAgent instance named "user_proxy"

user_proxy = autogen.UserProxyAgent(
    name                       = "user_proxy",
    human_input_mode           = "NEVER", # NEVER / ALWAYS / TERMINATE
    max_consecutive_auto_reply = 10,

    # if the x["content"] ends by "TERMINATE", is_termination_msg-->True; otherwise, is_termination_msg--> False
    is_termination_msg         = lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    
    code_execution_config = {
        "work_dir": "coding",
        
        # Using docker is safer than running the generated code directly.
        # set use_docker=True if docker is available to run the generated code. 
        "use_docker": False
    },
    
    llm_config=llm_config
)

user_proxy

In [16]:
# create an AssistantAgent named "assistant"

assistant = autogen.AssistantAgent(
    name       = "assistant",
    llm_config = llm_config
)

assistant

In [17]:
%%time
# the assistant receives a message from the user_proxy, which contains the task description in the "message" field

chat_res = user_proxy.initiate_chat(
    assistant,
    message = """What date is today? Compare the year-to-date gain for META and TESLA.""",
    summary_method = "reflection_with_llm",
)

user_proxy (to assistant):

What date is today? Compare the year-to-date gain for META and TESLA.

--------------------------------------------------------------------------------
assistant (to user_proxy):

First, let's find out today's date using Python. After that, we will proceed to compare the year-to-date (YTD) gain for META (Facebook's parent company, Meta Platforms, Inc.) and TESLA (Tesla, Inc.).

```python
from datetime import datetime

# Print the current date
print(datetime.now().strftime('%Y-%m-%d'))
```

Please run this code to get today's date. After that, we will move on to the next step of comparing the YTD gains for META and TESLA.

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
2024-04-25


--------------------------------------------------------------------------------
assistant (to user_proxy)

KeyboardInterrupt: 

## Only if needed:

In [8]:
%%time

# followup of the previous question, if needed
user_proxy.send(
    recipient=assistant,
    message="""Please try again and use another way to gather the stock price. For example, use yfinance.""",
)

user_proxy (to assistant):

Please try again and use another way to gather the stock price. For example, use yfinance.

--------------------------------------------------------------------------------
assistant (to user_proxy):

It appears there was a misunderstanding in the interpretation of the task. The previous response did use `yfinance` to gather the stock prices and calculate the year-to-date (YTD) gains for META and TESLA. The output provided was based on the data fetched from Yahoo Finance via the `yfinance` library, which is a common method to programmatically access stock price information.

The YTD gains calculated were:
- **META (Meta Platforms, Inc.)** had a YTD gain of **41.21%**.
- **TESLA (Tesla, Inc.)** had a YTD loss of **-42.15%**.

This calculation was done by comparing the stock prices from the beginning of the year to the most recent closing price available (as of the date the script was run). The `yfinance` library fetched historical data for each stock, from wh

## Let's check the History

In [9]:
i = 1
for cr in chat_res.chat_history:
    print(f'Step {i} --> {cr["content"].rstrip()[-9:]}')
    i=i+1

Step 1 --> nd TESLA.
Step 2 --> nd TESLA.
Step 3 --> 024-04-24
Step 4 --> nd TESLA.
Step 5 --> 2024.2.2)
Step 6 --> TERMINATE
Step 7 --> yfinance.
Step 8 --> nd TESLA.
Step 9 --> e to ask!
Step 10 -->  further!
Step 11 --> sist you.
Step 12 --> TERMINATE


In [10]:
# read the content of the cache

import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect(f'./.cache/{cache_seed}/cache.db')

# Create a cursor object
cur = conn.cursor()

# List all tables in the database
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(f"Tables in this DB: {cur.fetchall()}\n")

# Execute a query
cur.execute("SELECT * FROM Cache")

# Fetch and print the results
for row in cur.fetchall():
    print(row)

# Close the connection
conn.close()

Tables in this DB: [('Settings',), ('Cache',)]

(1, '{"messages": [{"content": "You are a helpful assistant.", "role": "system"}, {"content": "You\'re a retrieve augmented chatbot. You answer user\'s questions based on your own knowledge and the\\ncontext provided by the user.\\nIf you can\'t answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.\\nYou must give as short an answer as possible.\\n\\nUser\'s question is: What is autogen?\\n\\nContext is: # Harnessing the Power of AutoGen and OpenAI GPTs for Advanced Code Interpretation and Development | by Dr. Ernesto Lee | Medium\\n\\nHarnessing the Power of AutoGen and OpenAI GPTs for Advanced Code Interpretation and Development | by Dr. Ernesto Lee | Medium[Open in app](https://rsci.app.link/?%24canonical_url=https%3A%2F%2Fmedium.com%2Fp%2F571ddb6f814c&%7Efeature=LoOpenInAppButton&%7Echannel=ShowPostUnderUser&source=---two_column_layout_nav----------------------------------)[Sign up](https: